<a href="https://colab.research.google.com/github/swlee23/Deep-Learning-Elevator-Current-Time-Series-Classification/blob/master/1D_CNN_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Things to be included
# 1. visualization tools for optimization
# 2. visualization tools for plotting actual and predicted sequence, and anomaly points
# 3. Computation cost calculation
# 4. Warnings / Errors?

In [0]:
import numpy as np
from numpy import array
import random
from random import randint
import os
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Activation, MaxPooling1D, Dropout
from keras.optimizers import SGD

In [0]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1" #model will be trained on GPU 1

In [0]:
                '''Parameters'''
### Parameters
num_data = 47
  
### Model Hyperparameters
w = 2000                 # History window (number of time stamps taken into account) 
                         # i.e., filter(kernel) size       
p_w = 300                # Prediction window (number of time stampes required to be 
                         # predicted)
n_features = 1           # Univariate time series
num_category = 4
category_dict = {0 : 'UP_1-2F', 1: 'UP_3-4F', 2 : 'UP_5-7F', 3 : 'DOWN_1-2F', 4: 'DOWN_3-4F', 5 : 'DOWN_5-7F'}

kernel_size = 3          # Size of filter in conv layers
num_neuron_1 = 32        # Number of filters in first conv layer
num_neuron_2 = 32        # Number of filters in second conv layer
num_neuron_dl = 40       # Number of neurons in dense layer

conv_strides = 1
pool_size_1 = 2          # Length of window of pooling layer 1
pool_size_2 = 2          # Length of window of pooling layer 2
pool_strides_1 = 2       # Stride of window of pooling layer 1
pool_strides_2 = 2       # Stride of window of pooling layer 2

epochs = 30
learning_rate = 2e-5  

In [0]:
                """Data loading"""
data_value_dict = {}
data_label_dict = {}
max_length = 0
for i in range(1, num_data+1):
  df_temp = pd.read_csv('data/csv/'+str(i)+'.csv',
                         names = ['DataSavedTime', 'Item005', 'Label'], skiprows = [0])
  data_value_dict[i] = list(df_temp['Item005'])
  data_label_dict[i] = df_temp['Label'][0]
  if (len(df_temp['Item005']) > max_length):
    max_length = len(df_temp['Item005'])

In [16]:
                """Data preprocessing"""
  
# Map raw labels to each category
def is_up(label):
    return (label / 100 >= 1)
def is_0_2F(label):
    if (is_up(label)) :
      label = label - 100
    return (label / 10 < 3)
def is_3_4F(label):
    if (is_up(label)) :
      label = label - 100
    return (3 <= label / 10 < 5)

for i in range(1, num_data+1):
  print (data_label_dict[i])
  label = data_label_dict[i]
  if is_up(label) :
    if is_0_2F(label) : data_label_dict[i] = 0
    elif is_3_4F(label) : data_label_dict[i] = 1
    else : data_label_dict[i] = 2
  else :
    if is_0_2F(label) : data_label_dict[i] = 3
    elif is_3_4F(label) : data_label_dict[i] = 4
    else : data_label_dict[i] = 5
  print (data_label_dict[i])
  print ('---------')
  
  
# Pad time series to make them equal in length
for i in range(1, num_data+1):
  ser_len = len(data_value_dict[i])
  if ser_len < max_length :
    pad_ratio = (max_length-ser_len)
    # print(pad_ratio, ser_len)
  else :
    continue


161.0
2
---------
61.0
5
---------
60.0
5
---------
141.0
1
---------
120.0
0
---------
151.0
2
---------
111.0
0
---------
61.0
5
---------
161.0
2
---------
60.0
5
---------
20.0
3
---------
160.0
2
---------
160.0
2
---------
60.0
5
---------
60.0
5
---------
10.0
3
---------
111.0
0
---------
142.0
1
---------
111.0
0
---------
110.0
0
---------
20.0
3
---------
40.0
4
---------
140.0
1
---------
150.0
2
---------
60.0
5
---------
140.0
1
---------
110.0
0
---------
110.0
0
---------
62.0
5
---------
60.0
5
---------
140.0
1
---------
60.0
5
---------
160.0
2
---------
161.0
2
---------
32.0
4
---------
32.0
4
---------
162.0
2
---------
160.0
2
---------
70.0
5
---------
110.0
0
---------
121.0
0
---------
140.0
1
---------
60.0
5
---------
60.0
5
---------
161.0
2
---------
160.0
2
---------
160.0
2
---------


In [0]:
# Split data into training set and test set
test_ix = randint(1,num_data)
X_test = data_value_dict[test_ix]
Y_test = data_label_dict[test_ix]
if (test_ix != num_data) :
  temp_label = data_label_dict[num_data]
  temp_val = data_value_dict[num_data]
  data_label_dict[num_data] = data_label_dict[test_ix]
  data_value_dict[num_data] = data_value_dict[test_ix]
  data_label_dict[test_ix] = temp_label
  data_value_dict[test_ix] = temp_val
  del data_value_dict[num_data]
  del data_label_dict[num_data]
else :
  del data_value_dict[num_data]
  del data_label_dict[num_data]
X_train = data_value_dict.values()

#reshaping
#this assumes our data format
#For 3D data, "channels_last" assumes (conv_dim1, conv_dim2, conv_dim3, channels) while 
#"channels_first" assumes (channels, conv_dim1, conv_dim2, conv_dim3).
if k.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
#more reshaping
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape) #X_train shape: (60000, 28, 28, 1)

In [0]:
                  '''Model building'''
model = Sequential()
model.add(Conv1D(num_neuron_1, kernel_size=kernel_size,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(num_neuron_2, kernel_size=kernel_size, activation='relu'))
model.add(MaxPooling1D(pool_size=pool_size)
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(num_neuron_dl, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_category, activation='softmax'))

In [0]:
                 '''configure model'''
model.compile(optimizer='adam', 
              loss=keras.losses.categorical_crossentropy)

# sgd = keras.optimizers.SGD(lr=learning_rate, 
#                          decay=1e-6, 
#                          momentum=0.9, 
#                          nesterov=True)
# model.compile(optimizer='sgd', 
#               loss='mean_absolute_error', 
#               metrics=['accuracy'])

W0704 08:34:53.491716 140133400999808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
                    '''Training'''
model_log = model.fit(X_train, y_train,
                      batch_size=batch_size,
                      epochs=num_epoch,
                      verbose=1,
                      validation_data=(X_test, y_test))

W0704 08:34:53.880914 140133400999808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/30
2702/2702 [==============================] - 7s 3ms/step - loss: 0.4650
Epoch 2/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2884
Epoch 3/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2483
Epoch 4/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2328
Epoch 5/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2306
Epoch 6/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2253
Epoch 7/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2266
Epoch 8/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2231
Epoch 9/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2230
Epoch 10/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2247
Epoch 11/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2226
Epoch 12/30
2702/2702 [==============================] - 6s 2ms/step - loss: 0.2255
E

In [0]:
# plotting the metrics
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(model_log.history['acc'])
plt.plot(model_log.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.subplot(2,1,2)
plt.plot(model_log.history['loss'])
plt.plot(model_log.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.tight_layout()
fig

In [0]:
                 """Testing with random interval"""


1/1 [==============================] - 0s 46ms/step


0.11631018668413162

In [0]:
               """Save Weights (DeepAnT)"""
# save it to disk so we can load it back up anytime
model.save_weights('sinwave_DeepAnT_1.h5')  

In [0]:
            """Classifying test sequence""" 
# Build model 
model = Sequential()
model.add(Conv1D(filters=num_filt_1,
                 kernel_size=kernel_size,
                 strides=conv_strides,
                 padding='valid',
                 activation='relu',
                 input_shape=(w, n_features)))
model.add(MaxPooling1D(pool_size=pool_size_1)) 
model.add(Conv1D(filters=num_filt_2,
                 kernel_size=kernel_size,
                 strides=conv_strides,
                 padding='valid',
                 activation='relu'))
model.add(MaxPooling1D(pool_size=pool_size_2))
model.add(Flatten())
model.add(Dense(units=num_nrn_dl, activation='relu')) 
model.add(Dropout(dropout_rate))
model.add(Dense(units=num_nrn_ol))

# Load the model's saved weights.
model.load_weights('sinwave_DeepAnT_1.h5')
          
# Sample a portion of the raw_seq randomly
# 1. Choose 
ran_ix = random.randint(1,len(raw_seq) - w - p_w)
input_seq = array(raw_seq[ran_ix : ran_ix + w])
target_seq = array(raw_seq[ran_ix + w : ran_ix + w + p_w])
input_seq = input_seq.reshape((1, w, n_features))

# Predict the next time stampes of the sampled sequence
yhat = model.predict(input_seq, verbose=1)

# Print our model's predictions.
print(yhat)

# Check our predictions against the ground truths.
print(target_seq) # [7, 2, 1, 0, 4]

1/1 [==============================] - 0s 49ms/step
[[ 5.3346670e-01  5.7779747e-01  6.1173660e-01  6.5328515e-01
   6.8654871e-01  7.2793227e-01  7.4849910e-01  7.7159363e-01
   7.9978532e-01  8.2604152e-01  8.3773297e-01  8.5419333e-01
   8.6456931e-01  8.7446964e-01  8.7559789e-01  8.8003802e-01
   8.7642407e-01  8.7025601e-01  8.6118495e-01  8.5479534e-01
   8.3706415e-01  8.1483984e-01  8.0458981e-01  7.7464014e-01
   7.4558175e-01  7.0788634e-01  6.7713135e-01  6.3544267e-01
   5.8623677e-01  5.5745482e-01  5.1033270e-01  4.6085200e-01
   4.2107847e-01  3.6753997e-01  3.1533259e-01  2.5496286e-01
   2.1251279e-01  1.6309837e-01  1.0952963e-01  5.0019491e-02
   1.9704960e-03 -5.3943742e-02 -1.0688288e-01 -1.6401196e-01
  -2.1575488e-01 -2.8042656e-01 -3.4290662e-01 -3.9345095e-01
  -4.3836045e-01 -4.9260610e-01 -5.3311098e-01 -5.7782698e-01
  -6.2342823e-01 -6.5971470e-01 -7.0389074e-01 -7.2249669e-01
  -7.4808878e-01 -7.7117854e-01 -8.0327076e-01 -8.2011247e-01
  -8.3632350e-01 -

In [0]:
# Shallow CNN version  
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [-5, 5, -10, 10, -15, 15, -20, 20, -25]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mean_absolute_error')
# fit model
model.fit(X, y, epochs=500, verbose=1)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=1)
print(yhat)

Epoch 1/500
5/5 [==============================] - 0s 57ms/step - loss: 17.8328
Epoch 2/500
5/5 [==============================] - 0s 432us/step - loss: 17.7493
Epoch 3/500
5/5 [==============================] - 0s 560us/step - loss: 17.6573
Epoch 4/500
5/5 [==============================] - 0s 1ms/step - loss: 17.5538
Epoch 5/500
5/5 [==============================] - 0s 513us/step - loss: 17.4611
Epoch 6/500
5/5 [==============================] - 0s 542us/step - loss: 17.3709
Epoch 7/500
5/5 [==============================] - 0s 533us/step - loss: 17.2777
Epoch 8/500
5/5 [==============================] - 0s 586us/step - loss: 17.1848
Epoch 9/500
5/5 [==============================] - 0s 574us/step - loss: 17.0935
Epoch 10/500
5/5 [==============================] - 0s 537us/step - loss: 17.0046
Epoch 11/500
5/5 [==============================] - 0s 516us/step - loss: 16.9190
Epoch 12/500
5/5 [==============================] - 0s 606us/step - loss: 16.8386
Epoch 13/500
5/5 [==========

In [0]:
               """Save Weights (ShallowAnT)"""
# save it to disk so we can load it back up anytime
model.save_weights('sinwave_ShallowAnT_1.h5')  